In [2]:
# Importing required Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# importing USDA data

FOOD_DES = pd.read_csv('../USDA/FOOD_DES.csv')
NUT_DATA = pd.read_csv('../USDA/NUT_DATA.csv')
WEIGHT = pd.read_csv('../USDA/Weight-out.csv')
NUTR_DEF = pd.read_csv('../USDA/NUTR_DEF.csv')

/home/jushaan/Downloads/yes/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
'''clarifying some column names in NUTR_DEF where number:number appears in column name, 
here we need to add column name above columns with :
'''


l = list(NUTR_DEF['description'].values) #list of description names
t = 0
for i in range(len(l)):

    if (':'in l[i]) and (i>t) and t<len(l):
        x = l[i-1]
        for j in range(i,len(l)):
            if ':' in l[j]:
                l[j]=x+" "+l[j]
                t = j
            else:
                t = j
                break
NUTR_DEF['description'] = l

In [3]:
# merging data on nutritient number

NUT_CPLT = pd.merge(NUT_DATA,NUTR_DEF, how ='inner', on = ['nutrient_no'])   

# dropping columns not being used

NUT_CPLT.drop(['num_data_points','std_err','source_cd','deriv_cd','ref_ndb_no','add_nutr_mark','num_studies','low_err','upper_err','stat_comment','add_mod_date','conf_code','min_val','max_val','df'], axis =1,inplace = True)

# merging some more data to get complete nutrient profile

NUT_CPLT = pd.merge(NUT_CPLT,WEIGHT, how ='inner', on =['ndb_no'])
NUT_CPLT.drop(['num_dec','sr_order'], axis = 1, inplace = True)
NUT_CPLT.rename({'ndb_no':'ndb_id'},inplace = True,axis = 1)
NUT_CPLT.head()

del FOOD_DES
del NUT_DATA
del NUTR_DEF
del WEIGHT

In [4]:
# Cleaning more data in this cell

def func_1(s):
    l =  s.split('(')
    l[0] = l[0].strip()
    return l[0]

NUT_CPLT.loc[NUT_CPLT['measure_desc'].str.contains('\('),'measure_desc']= NUT_CPLT.loc[NUT_CPLT['measure_desc'].str.contains('\('),'measure_desc'].apply(func_1)
NUT_CPLT.rename(columns={'measure_desc': 'unit'}, inplace=True)

In [4]:
recipe=pd.read_csv('../Recipes-file-edited/Recipe_correct_ndb.csv')
recipe.head()

,recipe_no,ingredient_Phrase,ingredient,state,quantity,unit,temp,df,size,ing_id,ndb_id,M_or_A
0,2610,3 cups water,water,NaN,3,cups,NaN,NaN,NaN,3,14555,M
1,2610,1 cup red lentils,red lentil,NaN,1,cup,NaN,NaN,NaN,452,16144,A
2,2610,"1 roma tomato , quartered",rom tomato,quartered,1,NaN,NaN,NaN,NaN,180,93600,A
3,2610,"1 carrot , quartered",carrot,quartered,1,NaN,NaN,NaN,NaN,21,11124,M
4,2610,"1 small onion , quartered",onion,quartered,1,NaN,NaN,NaN,small,1,11282,M


In [6]:
# fixing some errors in the file

recipe.loc[recipe['ndb_id']=='9150 / 9156','ndb_id']=9150
recipe.loc[recipe['ndb_id']=='"09252\t"','ndb_id'] = 9252
recipe.loc[recipe['ndb_id']=='21523/ 21524','ndb_id']=0
recipe.loc[recipe['ndb_id']=='caramel ice cream topping','ndb_id']=0
recipe.loc[recipe['ndb_id']=='12156','ndb_id']=12155

/home/jushaan/Downloads/yes/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [7]:
# fill measuring description where required with "unknown"

recipe['unit'].fillna( recipe['size'], inplace = True)
recipe['unit'].fillna( "unknown", inplace = True)

#list of recipe numbers

recipeno = [x for x in recipe["ndb_id"].values]
recipe['quantity'] = recipe['quantity'].str.replace('⁄','/')

In [8]:

# cleaning measurement method using lemmatizer

import nltk
import re
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    s =""
    l = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
    for x in l:
        s +=" " + x
        break # Taking only the first name 
    s = re.sub('[^A-Za-z ]', '', s)
    s = s.strip()
    return s

# applying above function to both Tables we have

recipe['unit'] = recipe.unit.apply(lemmatize_text)
NUT_CPLT['unit'] = NUT_CPLT.unit.apply(lemmatize_text)

# replacing commas by space in NUTRITION TABLE

NUT_CPLT["unit"] = NUT_CPLT["unit"].str.replace(","," ")
recipe['unit'].replace('', np.nan, inplace=True) # To replce these by size and then unknown
recipe['unit'].fillna(recipe['size'], inplace = True)
recipe['unit'].fillna('unknown', inplace = True)

In [13]:

#cleaning data

recipe['quantity'] = recipe['quantity'].str.strip()

# MAKING SURE NDB_NO IS OF TYPE INT

recipe["ndb_id"] =recipe["ndb_id"].astype(int)

# MAKING SHORT FORMS TO MATCH unit in both tables
short_forms={'teaspoon':'tsp','tablespoon':'tbsp','fluid ounce':'fl oz','ounce':'oz','cup':'cup','pound':'lb','milliliter':'ml'}
for key,value in short_forms.items():
    recipe.loc[recipe["unit"].str.contains(key), 'unit'] = value
    NUT_CPLT.loc[NUT_CPLT["unit"].str.contains(key), 'unit'] = value


In [15]:
# cleaning entities missed by NER

recipe.loc[(recipe['ingredient_Phrase'].str.contains(' g ')),'unit']='g'
recipe.loc[(recipe['ingredient_Phrase'].str.contains(' ml ')),'unit']='ml'
unit_conv={'ounces':'oz','teaspoon':'tsp','tablespoon':'tbsp','cup':'cups','clove':'clove','lb':'lb','kg':'kg','litre':'liter','millilitre':'ml','gram':'g','g/':'g'}

for key,value in unit_conv.items():
       recipe.loc[(recipe['unit']=='unknown') & (recipe['ingredient_Phrase'].str.contains(key)),'unit']=value

        
recipe.loc[(recipe['unit']=='unknown') & (recipe['ingredient']=='garlic clove'),'unit']='clove'
recipe.loc[(recipe['unit']=='unknown') & (recipe['ingredient'].str.contains('lemon')) & (recipe['ingredient'].str.contains('grass')),'ndb_id']=11972
recipe.loc[(recipe['unit']=='unknown') & (recipe['ingredient'].str.contains('lemon')) & (recipe['ingredient'].str.contains('grass')),'unit']='tbsp'
recipe.loc[(recipe['unit']=='unknown') & (recipe['ingredient'].str.contains('lemon')),'unit']=''
recipe.loc[(recipe['unit']=='unknown') & (recipe['ndb_id']==9152),'unit']='lemon'
recipe.loc[(recipe['unit']=='unknown') & (recipe['ndb_id']==1150),'unit']='fruit'
recipe.loc[(recipe['unit']=='unknown') & (recipe['ndb_id']==11209),'unit']='eggplant'



recipe.loc[recipe['ingredient_Phrase'].str.contains('117915'),'quantity']=1

### NEXT COUPLE OF CELLS WILL BE USED TO CLEAN THE UNKNOWN MEASUREMENT IN RECIPES

In [16]:
import operator

from collections import Counter
recipe.loc[recipe['ingredient'].isna(),'ingredient']='unknown'

In [16]:
# # List of ingredient names when measure_desc is unknown
# x = recipe.loc[recipe['ingredient']=='carrot']['unit']
# l_unk = list(x)

# count = Counter()
# for word in l_unk:
#     count[word]+=1
# l_unk = count

# sorted_x_3 = sorted(l_unk.items(), key=operator.itemgetter(1))
# sorted_x_3.reverse()
# sorted_x_3

In [17]:
# List of ingredient names when measure_desc is unknown

x = recipe[recipe['unit']=='unknown']['ingredient']
l_unk = list(x)

count = Counter()
for word in l_unk:
    count[word]+=1
l_unk = count

sorted_x_3 = sorted(l_unk.items(), key=operator.itemgetter(1))
sorted_x_3.reverse()
sorted_x_3


[('egg', 18318),
 ('salt', 11093),
 ('onion', 10775),
 ('salt pepper', 10351),
 ('black pepper', 4796),
 ('bay leaf', 4767),
 ('pepper', 4638),
 ('tomato', 4095),
 ('green onion', 3682),
 ('carrot', 3648),
 ('salt black pepper', 3640),
 ('egg yolk', 2503),
 ('red bell pepper', 2238),
 ('lime', 2072),
 ('egg white', 1826),
 ('cinnamon', 1776),
 ('green bell pepper', 1728),
 ('potato', 1711),
 ('water', 1695),
 ('avocado', 1655),
 ('jalapeno pepper', 1637),
 ('flour tortilla', 1622),
 ('scallion', 1620),
 ('red onion', 1609),
 ('chicken breast', 1505),
 ('olive oil', 1479),
 ('oil', 1447),
 ('cream', 1421),
 ('shallot', 1366),
 ('green pepper', 1332),
 ('red pepper', 1270),
 ('corn tortilla', 1223),
 ('parsley', 1170),
 ('lime juice', 1097),
 ('celery rib', 1076),
 ('parmesan cheese', 1037),
 ('cucumber', 1016),
 ('sugar', 997),
 ('jalapeno', 985),
 ('cooking spray', 980),
 ('cilantro', 922),
 ('orange', 914),
 ('butter', 913),
 ('chicken', 892),
 ('sea salt', 891),
 ('kosher salt', 887)

In [ ]:
# Used to figure out most common measure_desc and updating recipe table accordingly
count =0
for ing_fre in sorted_x_3:
    ing,frq=ing_fre
    x = recipe[recipe['ingredient']==ing]['unit']
    li = list(x)

    from collections import Counter
    count1 = Counter()
    for words in li:
        count1[words]+=1
    li = count1

    import operator

    sorted_x = sorted(li.items(), key=operator.itemgetter(1))
    sorted_x.reverse()
    m='unknown'
    for meas in sorted_x:
        measurement,freq=meas
        m=measurement
        break
    recipe.loc[(recipe['unit']=='unknown') & (recipe['ingredient']==ing),'unit']=m
    count+=1
    if count%500==0:
        print(count)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000


In [ ]:
# creating list of ndb numbers in recipe
y = list(recipe['ndb_id'].values)
ndb_ls =[int(x) for x in y]


# filtering nutrient list based on ndb numbers
NUT_CPLT = NUT_CPLT[NUT_CPLT["ndb_id"].isin(ndb_ls)].reset_index() 
NUT_CPLT.drop(['index'], inplace = True, axis = 1)
NUT_CPLT["ndb_id"] = NUT_CPLT["ndb_id"].astype(int)
NUT_CPLT.head()

In [ ]:
recipe.to_csv('./recipe_measurements_edited.csv',index = None)
NUT_CPLT.to_csv('./nut_edited.csv',index = None)